# Assignment 2: JSON to CSV Conversion and Data Visualization
## Overview:

In this assignment, you'll:

Read a JSON file containing transportation data.
Convert the data to a CSV file.
Visualize the data using Matplotlib and Seaborn using the dataset output.

Dataset:

The dataset contains transportation information, including delay times, scheduled times, and route IDs. Each JSON file holds trip data.

## Step 0: Install and import libraries, Download Dataset

In [ ]:
import sys
!{sys.executable} -m pip install --upgrade tqdm
!{sys.executable} -m pip install --upgrade requests
!{sys.executable} -m pip install --upgrade pandas

In [ ]:
import requests
dataset_url = "https://gitea.zuzu.red/EthanPisani/LTC-Bus/media/branch/main/sample_data/day_dataset.zip"
r = requests.get(dataset_url, allow_redirects=True)
open('data/day_dataset.zip', 'wb').write(r.content)

# unzip the dataset
import zipfile

with zipfile.ZipFile("data/day_dataset.zip", 'r') as zip_ref:
    zip_ref.extractall("data")

# remove the zip file
import os
os.remove("data/day_dataset.zip")

In [5]:
import json
import csv
import pandas as pd
import glob
import time
import tqdm
import multiprocessing
from multiprocessing import Pool


## Part 1: Inspecting the data

In this section we will inpect the data and see what it does. Try to play arround with the json file and get data into a test csv file.

Data points such as: scheduled unix timestamp, route_id, delay, stop_id, vehicle_id

In [ ]:
data = None
unixTime = 1730113021 # timestamp in unix time that the snapshot was taken (in the file name)
# read the test file
with open('data/TripUpdate1730113021.json', 'r') as f:
    try:
        data = json.load(f)
    except ValueError:
        print("File is empty")
        sys.exit(1)

    #print
    # print(json.dumps(data, indent=4))
    output_json = [] # list of dict data to save to a file
    for entity in data.get("entity", []): # loop through each entity (is a bus update object)
        entity_id = entity.get("id", None)
        trip_update = entity.get("trip_update", {}) # data in the update
        vehicle_info = trip_update.get("vehicle", {}) # vehicle info
        trip = trip_update.get("trip", {})
        route_id = trip.get("route_id", None)
            

        # test prints, play arround with the data to see what is available
        print("Entity ID: ", json.dumps(entity_id, indent=4))
        print("Trip Update: ", json.dumps(trip_update, indent=4))
        print("Vehicle Info: ", json.dumps(vehicle_info, indent=4))
        
        # test output json
        output_json.append({
            "entity_id": entity_id,
            "route_id": route_id,      
        })

   
    # save output json to a csv file
    with open('output_test.csv', 'w') as f:
        writer = csv.DictWriter(f, fieldnames=output_json[0].keys())
        writer.writeheader()
        for row in output_json:
            writer.writerow(row)

# You can open the output in a new file at the bottom of the jupyter notebook output cell
# looks like this:
# Output is truncated. View as a scrollable element or open in a text editor. Adjust cell output settings...



## Part 2: Reading and Converting JSON to CSV

Read the JSON file. Use Python's json module to load data from a file.
Transform the data. Extract relevant fields like route_id, stop_id, delay, and scheduled_time.
Save to CSV. Use the csv module or Pandas to save the extracted data as a CSV file.

## No need to edit this function!


In [4]:
# Function to process files in parallel
def process_files(file_names):
    extracted_data = []
    seen_stops = set()
    stop_id_map = {}

    for file_name in tqdm.tqdm(file_names):
        time_stamp = int(file_name.split("TripUpdate")[1].split(".json")[0])

        # Read and process the JSON file
        with open(file_name, 'r') as file:
            try:
                data = json.load(file)
            except json.JSONDecodeError:
                print(f"Error reading file: {file_name}")
                continue

            for entity in data.get("entity", []):
                entity_id = entity.get("id", None)
                trip_update = entity.get("trip_update", {})
                vehicle_info = trip_update.get("vehicle", {})
                vehicle_id = vehicle_info.get("id", None) if vehicle_info else None

                stop_time_updates = trip_update.get("stop_time_update", [])
                trip = trip_update.get("trip", {})
                route_id = trip.get("route_id", "")

                for stop in stop_time_updates:
                    arrival_info = stop.get("arrival", {})
                    departure_info = stop.get("departure", {})

                    stop_id = stop.get("stop_id", None)

                    if arrival_info:
                        delay = arrival_info.get("delay", None)
                        scheduled_time = arrival_info.get("schedule_time", None)
                        actual_time = arrival_info.get("time", None)
                    elif departure_info:
                        delay = departure_info.get("delay", None)
                        scheduled_time = departure_info.get("schedule_time", None)
                        actual_time = departure_info.get("time", None)
                    else:
                        delay = None
                        scheduled_time = None
                        actual_time = None

                    if actual_time and time_stamp - 60 <= actual_time < time_stamp + 60:
                        # Efficiently convert times
                        ts = pd.to_datetime(scheduled_time, unit='s')
                        scheduled_time_second = ts.hour * 3600 + ts.minute * 60 + ts.second
                        ts2 = pd.to_datetime(actual_time, unit='s')
                        actual_time_second = ts2.hour * 3600 + ts2.minute * 60 + ts2.second

                        day_of_week = ts.weekday()
                        day_of_year = ts.dayofyear
                        if stop_id not in seen_stops:
                            seen_stops.add(stop_id)
                            stop_id_map[stop_id] = len(stop_id_map)

                        extracted_data.append({
                            "route_id": route_id,
                            "vehicle_id": vehicle_id,
                            "stop_id": stop_id_map[stop_id],
                            "delay": delay,
                            "scheduled_time": scheduled_time_second,
                            "actual_time": actual_time_second,
                            "day": day_of_week,
                            "day_of_year": day_of_year
                        })

    return extracted_data


## Step 2: Save data to CSV File

Write a function to save the dict as a csv file

In [5]:
def save_data_to_csv(extracted_data, file_name):
    # Save the extracted data to a CSV file


## Step 3: Run extraction

In [ ]:
dataset_folder = "data/day_dataset"
# Get all the files in the dataset folder
files = glob.glob(f"{dataset_folder}/*.json")
files.sort()

extracted_data = []

# this will take a while (5-20 minutes)
extracted_data = process_files(files)
# Save the extracted data to a CSV file
save_data_to_csv(extracted_data, "extracted_data.csv")
print("Data extraction complete!")

extracted_data = pd.read_csv("extracted_data.csv")
extracted_data


In [ ]:
# only route 9
extracted_data_9 = extracted_data[extracted_data["route_id"] == 9]
print(extracted_data_9.head())

## Part 2: Data Visualization

### Step 0: Import libraries

In [8]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:

# read data
trip_dataset = pd.read_csv("extracted_data.csv")

# remove nans
trip_dataset = trip_dataset.dropna()
trip_dataset


## Question 1: Analyze Delay Distribution for Each Route

Create a visualization to analyze the distribution of delays for each route_id. 
Use a boxplot to summarize delays and observe variations between routes.

Steps:

- Use a Seaborn boxplot to plot delay on the y-axis and route_id on the x-axis.
- Customize the plot with appropriate labels and a title for clarity.
- Add gridlines or style to enhance readability.

In [ ]:
# Create the boxplot using route_id and delay
plt.figure(figsize=(10, 6))



## Question 2: Compare Scheduled vs Actual Times

Create a scatter plot to compare scheduled_time and actual_time for all stops. Use different colors for each route_id to observe how routes differ in time deviations.

Steps:

-    Use Matplotlib's scatter function or Seaborn's scatterplot for the visualization.
-    Use only one route from the dataset
-    Create a new continuous numerical feature, "hour", that is the hour of the day with decmials. 2:30 AM would convert to 2.5 for example.
-    Plot hour on the x-axis and delay on the y-axis with an Alpha value that looks good.
-    Add labels and title

In [ ]:
# Create the scatter plot for route_id 10 delay over actual time, c
trip_dataset_10 = 
# create new feature for hour of the day
